In [ ]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import inference_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import pickle

In [ ]:
MODEL = 'res2Net_centernet2.py' # model config 경로
PATH = '../work_dirs/no-rigid-aug_centernet2_res2Net_pafpn_1x_coco/1' # 모델 저장된 폴더
EPOCH ='best_bbox_mAP_50_epoch_24.pth' # model checkpoint 파일이름

cfg =Config.fromfile(os.path.join(PATH,MODEL))

In [ ]:
# build dataset & dataloader

cfg.data.test.test_mode = True
test_dataset = build_dataset(cfg.data.test)
test_loader = build_dataloader(
        test_dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

In [ ]:
# build model

checkpoint_path = os.path.join(PATH,EPOCH)

cfg.data.test.test_mode = True
cfg.model.test_cfg.rpn.nms.type='soft_nms'
cfg.model.test_cfg.rcnn.nms.type='soft_nms'
#cfg.model.test_cfg.nms.type='soft_nms'

# build detector
model = build_detector(cfg.model) 
# ckpt load
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = test_dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [ ]:
# output 계산
output = single_gpu_test(model, test_loader, show_score_thr=0) 

In [ ]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['image_id'] = file_names
submission['PredictionString'] = prediction_strings
submission.to_csv(f'{PATH}/{PATH.split("/")[-1]}_submission.csv', index=None)
submission.head()

## Visualization

In [ ]:
import fiftyone as fo
from PIL import Image
from torchvision.transforms import functional as func
from mmdet.apis import inference_detector, show_result_pyplot
from PIL import Image
from torchvision.transforms import functional as func
import fiftyone as fo

### image 한 장 visualization
- 위에서 만들었던 submission 파일을 기반으로 visualization하는 것이 아니라, 하나씩 예측한 뒤 바로 visualization
- submission 파일 기반 visualization은 아래 __fiftyone__을 이용

In [ ]:
model.cfg = cfg
idx = 11

show_result_pyplot(model,test_dataset[idx]['img_metas'][0].data['filename'],output[idx])
# or save the visualization results to image files
#model.show_result(img, result, out_file='result.jpg')

# dataset visualization
- 위에서 만든 submission 파일 기반으로 visualization

In [ ]:
dataset = fo.Dataset.from_images_dir(images_dir='/opt/ml/detection/dataset/test')

In [ ]:
session = fo.launch_app(dataset) ## unlabeled dataset visualization

In [ ]:
predictions_view = dataset.limit(len(dataset))

# Add predictions to samples
with fo.ProgressBar() as pb:
    for idx,sample in enumerate(pb(predictions_view)):
        c, h, w = 3,1024,1024

        #result = inference_detector(model, sample.filepath)
        #show_result_pyplot(model, sample.filepath, result)
        result = output[idx]
        labels=[]
        scores=[]
        boxes=[]
        for label,values in enumerate(result):
            for pred in values:
                labels.append(label)
                scores.append(pred[-1])
                boxes.append(list(pred[0:4]))

        # Convert detections to FiftyOne format
        detections = []
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            x1, y1, x2, y2 = box
            rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

            detections.append(
                fo.Detection(
                    label=model.module.CLASSES[label],
                    bounding_box=rel_box,
                    confidence=score
                )
            )

        # Save predictions to dataset
        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

In [ ]:
session.view = predictions_view
session.freeze()